In [ ]:
#Import dataset

import pandas as pd
Data_original=pd.read_csv('C:\\Users\\Gianmarco\\Downloads\\Foundations of Computer Science\\googleplaystore.csv', encoding='UTF-8')



In [ ]:
# 1. Convert the app sizes to a number

for i in range(0,len(Data_original['Size'])):
    if ('M' in Data_original['Size'][i]):
        Data_original['Size'][i]=float(Data_original['Size'][i].replace('M',''))*1000000
    else:
        Data_original['Size'][i]=0
print(Data_original['Size'])


In [ ]:
# 2. Convert the number of installs to a number

for i in range(0,len(Data_original['Installs'])):
    Data_original['Installs'][i]=Data_original['Installs'][i].replace(',','')
    Data_original['Installs'][i]=Data_original['Installs'][i].replace('+','')
print(Data_original['Installs'])

In [ ]:
for i in range(0,len(Data_original['Size'])):
    if (Data_original['Size'][i]==0):
        Data_original['Size'][i]=None
print(Data_original['Size'])

In [ ]:
# 3. Transform “Varies with device” into a missing value

for i in range(0,len(Data_original['Current Ver'])):
    if Data_original['Current Ver'][i]=='Varies with device':
        Data_original['Current Ver'][i]=None
print(Data_original['Current Ver'])


In [ ]:
# 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

for i in range(0,len(Data_original['Android Ver'])):        
    if not type(Data_original['Android Ver'][i])==float and not Data_original['Android Ver'][i]==None:
        Data_original['Android Ver'][i]=Data_original['Android Ver'][i].replace(' and up','')
        Data_original['Android Ver'][i]=Data_original['Android Ver'][i].replace('W','')
    if Data_original['Android Ver'][i]=='Varies with device' and not type(Data_original['Android Ver'][i])==float:
        Data_original['Android Ver'][i]=None
    

    

In [ ]:
# Uso dill per salvare le modifiche
import dill

In [ ]:
dill.dump_session('notebook_session.db')

In [ ]:
dill.load_session('notebook_session.db')

In [ ]:
len(Data_original)

In [ ]:
# 5. Remove the duplicates

Data_original.drop_duplicates(inplace=True)

In [ ]:
len(Data_original)

In [ ]:
# 6. For each category, compute the number of apps
Data_original['Category'].value_counts()

In [ ]:
# 7. For each category, compute the average rating
Data_original.groupby('Category')['Rating'].mean()

In [ ]:
# 9. For each genre, create a new column of the original dataframe. 
#The new columns must have boolean values (True if the app has a given genre)

for i in range(0,len(Data_original['Category'])):
    if Data_original['Genres'].iloc[i] not in list(Data_original):
        if ';' in Data_original['Genres'].iloc[i]:
            generi=Data_original['Genres'].iloc[i].split(';')
            nome0=generi[0]
            nome1=generi[1]
            Data_original[nome0]=Data_original['Genres']==nome0
            Data_original[nome1]=Data_original['Genres']==nome1
        else:        
            nome=Data_original['Genres'].iloc[i]
            Data_original[nome]=Data_original['Genres']==nome
    print(i)
    

In [ ]:
list(Data_original)

In [ ]:
dill.dump_session('notebook_session.db')

In [ ]:
# Creo un nuovo dataframe
df_ = pd.DataFrame()
df_ = df_.fillna(0)

In [ ]:
# 10. For each genre, compute the average rating. What is the genre with highest average?
Data_original.groupby('Genres')['Rating'].mean()
Data_original.groupby('Genres')['Rating'].mean().idxmax()
# Non è corretto perchè molte app sono caratterizzate da due generi separati dal ';' , dunque tramite un ciclo for
# duplico ogni doppio genere,assegnando lo stesso rating. Il tutto aggiungendo le osservazioni al nuovo dataframe.

In [ ]:
for i in range(0,len(Data_original['Category'])):
    if ';' in Data_original['Genres'].iloc[i]:
        generi=Data_original['Genres'].iloc[i].split(';')
        df_=df_.append({'Genres':generi[0],'Rating':Data_original['Rating'].iloc[i]},ignore_index=True)
        df_=df_.append({'Genres':generi[1],'Rating':Data_original['Rating'].iloc[i]},ignore_index=True)
    else:
        df_=df_.append({'Genres':Data_original['Genres'].iloc[i],'Rating':Data_original['Rating'].iloc[i]},ignore_index=True)

In [ ]:
# Ora calcolo i rating medi per genere e il genere corrispondente al valore massimo.
df_.groupby('Genres')['Rating'].mean()
df_.groupby('Genres')['Rating'].mean().idxmax()

In [ ]:
dill.dump_session('notebook_session.db')

In [ ]:
# Verifico la presenza di e ne scopro uno (genere sbagliato e rating pari a 19, la elimino)
df_[df_['Rating']>5]


In [ ]:
df_=df_.drop([10448],axis=0)

In [ ]:
df_.groupby('Genres')['Rating'].mean()

In [ ]:
df_.groupby('Genres')['Rating'].mean().idxmax()

In [ ]:
# Creo una nuova variabile 'Approx Income'
Data_original['Approx Income']=0

In [ ]:
list(Data_original['Price'])
# Vedo che le app non gratuite hanno il prezzo espresso in dollari. Dunque rimuovo il simbolo '$'

In [ ]:
for i in range(0,len(Data_original['Price'])):
    if '$' in Data_original['Price'].iloc[i]:
        Data_original['Price'].iloc[i]=Data_original['Price'].iloc[i].replace('$','')
        
    

In [ ]:
# Converto i prezzi da stringa a float prima di moltiplicarli per il numero di installazioni.
for i in range(0,len(Data_original['Price'])):
    if type(Data_original['Price'].iloc[i])!=str:
        Data_original['Price'].iloc[i]=float(Data_original['Price'].iloc[i])
    else:
        Data_original=Data_original.drop(Data_original.index[i])
    print(i)

In [ ]:
# 11. For each app, compute the approximate income, obtain as a product of number of installs and price.
for i in range(0,len(Data_original['Price'])):
    if (type(Data_original['Price'].iloc[i])!=str and type(Data_original['Installs'].iloc[i])!=str):
        Data_original['Approx Income'].iloc[i]=(Data_original['Price'].iloc[i])*(Data_original['Installs'].iloc[i])
    print(i)
    

In [ ]:
list(Data_original['Approx Income'])

In [ ]:
# 12. For each app, compute its minimum and maximum Sentiment_polarity
# Importo il dataset con le reviews
Data_reviews=pd.read_csv('C:\\Users\\Gianmarco\\Downloads\\Foundations of Computer Science\\googleplaystore_user_reviews.csv', encoding='UTF-8')

In [ ]:
Data_reviews.groupby('App')['Sentiment_Polarity'].min()
Data_reviews.groupby('App')['Sentiment_Polarity'].max()